In [ ]:
## import the necessary libraries
import importlib
from thrift.transport import TTransport, TSocket
from thrift.protocol.TBinaryProtocol import TBinaryProtocol
SeekRPC = importlib.import_module('gen-py.seek_rpc.SeekRPC')
ttypes = importlib.import_module('gen-py.seek_rpc.ttypes')
constants = importlib.import_module('gen-py.seek_rpc.constants')

In [ ]:
## establish a server connection
host = 'localhost'
port = 9090

socket = TSocket.TSocket(host, port)
transport = TTransport.TBufferedTransport(socket)
protocol = TBinaryProtocol(transport)
client = SeekRPC.Client(protocol)
transport.open()
# Verify version of client and server match
version = client.getRpcVersion()
assert version == constants.RPCVersion

In [ ]:
## run a query
def runQuery(species, genes, useSymbols=False, numToPrint=10):
    params = SeekRPC.SeekQueryParams(
        distanceMeasure=ttypes.DistanceMeasure.ZScoreHubbinessCorrected,
        minQueryGenesFraction=0.5,
        minGenomeFraction=0.5,
        useGeneSymbols=useSymbols
    )

    genes = [gene.upper() for gene in genes]
    queryArgs = SeekRPC.SeekQueryArgs(species=species, genes=genes, parameters=params)
    
    result = client.seekQuery(queryArgs)
    if result.success is True:
        for i, gs in enumerate(result.geneScores):
            print(f'gene: {gs.name}, {gs.value}')
            if i > numToPrint: break

        for i, ds in enumerate(result.datasetWeights):
            print(f'dset: {ds.name}, {ds.value}')
            if i > numToPrint: break
    else:
        print(f'query error: {result.statusMsg}')


In [ ]:
# functions to run a set of queries in parallel
class Query(object):
    def __init__(self, species="Unknown", genes=None, useSymbols=False):
        self.species = species
        self.genes = genes
        self.useSymbols = useSymbols

def initQuery(query):
    params = SeekRPC.SeekQueryParams(
        distanceMeasure=ttypes.DistanceMeasure.ZScoreHubbinessCorrected,
        minQueryGenesFraction=0.5,
        minGenomeFraction=0.5,
        useGeneSymbols=query.useSymbols)

    query.genes = [gene.upper() for gene in query.genes]
    query = SeekRPC.SeekQueryArgs(species=query.species, 
                              genes=query.genes, 
                              parameters=params)
    task_id = client.seekQueryAsync(query)
    return task_id

def completeQuery(task_id, numToPrint=10):
    result = client.getSeekResult(task_id, block=True)
    if result.success is True:
        print(f'### {task_id} ###')
        for i, gs in enumerate(result.geneScores):
            print(f'gene: {gs.name}, {gs.value}')
            if i > numToPrint: break

        for i, ds in enumerate(result.datasetWeights):
            print(f'dset: {ds.name}, {ds.value}')
            if i > numToPrint: break
    else:
        print(f'query error: {result.statusMsg}')
    

In [ ]:
# Run series of queries asynchronously
queries = [Query('human', ['SMO', 'PTCH1', 'PTCH2', 'BOC'], True),
           Query('fly', ['35234', '35232']),
           Query('mouse', ['GLI1', 'GLI2', 'PTCH1'], True),
           Query('worm', ['ptc-1', 'smo-1', 'tra-1'], True),
           Query('yeast', ['FIG1', 'FUS1', 'FUS2'], True),
           Query('zebrafish', ['PTCH2', 'PTCH1', 'BOC'], True)]

taskIds = [initQuery(q) for q in queries]
for taskId in taskIds:
    completeQuery(taskId)
    

In [ ]:
species = 'fly'
genes = ['CG18094', 'CG10189']
# genes = [35234', '35232']
runQuery(species, genes, useSymbols=True)

In [ ]:
species = 'mouse'
genes = ['GLI1', 'GLI2', 'PTCH1']
# genes = ['14632', '14633', '19206']
runQuery(species, genes, useSymbols=True)

In [ ]:
species = 'human'
genes = ['SMO', 'PTCH1', 'PTCH2', 'BOC']
runQuery(species, genes, useSymbols=True)

In [ ]:
species = 'sampleBC'
genes = ['55755', '64859', '348654', '79791', '7756', '8555', '835', '5347']
runQuery(species, genes, useSymbols=False)

In [ ]:
query = Query('sampleBC', ['55755', '64859', '348654', '79791', '7756', '8555', '835', '5347'], False)
taskId = initQuery(query)
completeQuery(taskId)

In [ ]:
species = 'mock'
genes = ['90634', '23659']
runQuery(species, genes, useSymbols=False)

In [ ]:
# Run series of queries asynchronously
queries = [Query('mock', ['90634', '23659']),
           Query('mock', ['90634', '23659']),
           Query('mock', ['90634', '23659']),
           Query('mock', ['90634', '23659']),
           Query('mock', ['90634', '23659']),
           Query('mock', ['90634', '23659']),
           Query('mock', ['90634', '23659']),
          ]

taskIds = [initQuery(q) for q in queries]
for taskId in taskIds:
    completeQuery(taskId)

In [ ]:
## close the server connection
transport.close()